In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import torch

if 'has_been_executed' not in locals():
    print("Changing directory to the root of the project")
    os.chdir("../../../")
    has_been_executed = True
else:
    print("The directory is correct")

# Sequence Manager

In [ ]:
from thesis.data_management.sequence_manager import SequenceManager
from matplotlib import pyplot as plt
from thesis.constants import SERIALS, TRAIN_CAMS, TEST_CAMS

sequence_manager = SequenceManager(3, cameras=list(range(16)))
intrinsics, extrinsics, names = sequence_manager.cameras
print(f"Intrinsics shape: {intrinsics.shape}")
print(f"Extrinsics shape: {extrinsics.shape}")
print(f"Names: {names}")
color_correction = sequence_manager.color_correction
print(f"Color correction shape: {color_correction.shape}")
audio, sr = sequence_manager.load_audio()
print(f"Audio shape: {audio.shape}")
print(f"Sample rate: {sr}")

if False:
    for i in range(16):
        image = sequence_manager.images[100, i]
        plt.imshow(image)
        title = f"Camera {i}"
        if i in TRAIN_CAMS:
            title += " (train)"
        elif i in TEST_CAMS:
            title += " (test)"
        plt.title(title)
        plt.show()

In [ ]:
# multi sequence manager
from thesis.data_management.sequence_manager import MultiSequenceManager
from thesis.constants import TRAIN_SEQUENCES

msm = MultiSequenceManager(TRAIN_SEQUENCES, cameras=list(range(16)))
print(f"Number of frames: {len(msm):,}")
msm[0].images[0, 0].shape

# Dataset

In [ ]:
from thesis.data_management.datasets import SingleSequenceDataset
from torch.utils.data.dataloader import DataLoader

ssd = SingleSequenceDataset(sequence=3, n_cameras_per_frame=1)

dataloader = DataLoader(
    ssd,
    batch_size=None,
    num_workers=12,
    persistent_workers=True,
)

In [ ]:
a = ssd[0]

In [ ]:
a.se3_transform.rotation.shape

In [ ]:
from tqdm import tqdm


def time_data_loader(data_loader, n_steps=1_000):
    iterator = iter(data_loader)
    for _ in tqdm(range(n_steps)):
        try:
            data = next(iterator)
        except StopIteration:
            iterator = iter(data_loader)
            data = next(iterator)
        data = data_loader.dataset.prepare_data(data)


time_data_loader(dataloader, 100)

## Quantization Dataset

In [ ]:
from thesis.data_management.datasets import QuantizationDataset
from torch.utils.data.dataloader import DataLoader
from thesis.constants import TRAIN_SEQUENCES

qd = QuantizationDataset(sequences=[3])

dataloader = DataLoader(
    qd,
    batch_size=32,
    num_workers=10,
    persistent_workers=True,
)
a = qd[0]

In [ ]:
len(dataloader)

In [ ]:
time_data_loader(dataloader, 1000)

In [ ]:
a = next(iter(dataloader))